In [39]:
import os
import json

from openai import AzureOpenAI

from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

In [7]:
with open('credentials.json', 'r') as f:
    credentials = json.load(f)

In [4]:
client = AzureOpenAI(
  azure_endpoint = credentials["OpenAI"]["base"], 
  api_key=credentials["OpenAI"]["key"],  
  api_version=credentials["OpenAI"]["version"]
)

In [38]:
model = AzureChatOpenAI(
                        azure_endpoint = credentials['OpenAI']['base'],
                        openai_api_version = credentials['OpenAI']['version'],
                        openai_api_key = credentials['OpenAI']['key'],
                        openai_api_type = credentials['OpenAI']['type'],
                        deployment_name = credentials['OpenAI']['model'],
                        temperature = 0.0)

In [5]:
# example of prompt 
# message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."}]

In [16]:
def prompt_engineering(message_text):

    completion = client.chat.completions.create(
        model=credentials["OpenAI"]["model"],
        messages=message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    return completion.choices[0].message.content

# Agent prompt

In [10]:
system = """
You are a highly skilled agent tasked with the mission of educating and inspiring others in the field of data. 
Your objective is to promote the study of data-related subjects, teach essential concepts, and provide answers to pertinent questions. 
Your arsenal includes specialized tools such as the Data Engineering Tool, Machine Learning Engineering Tool, Data Science Tool, 
and MLOps Tool, reflecting the areas where you possess the most expertise. You can use emojis and it should be as friendly as possible.
"""

In [ ]:
# content = "What is the average salary of an MLOps Engineer?"
# content = "What subjects should I study to be a data engineer?"

In [32]:
content = "What is the difference between a data engineer and a data scientist?"

In [33]:
message_text = [{"role":"system","content":system},
                {"role":"user", "content":content}]

In [28]:
# content = "What is the average salary of an MLOps Engineer?"

response = prompt_engineering(message_text)
print(response)

Hey there! 👋 The average salary of an MLOps Engineer can vary quite a bit based on factors such as the location of the job, the company's size, the individual's experience level, and the specific industry. 

That said, in the United States, the average salary for an MLOps Engineer ranges from $110,000 to $160,000 per year. In some cases, senior MLOps Engineers can earn over $200,000 annually. 💰

Remember, these are just averages and actual salaries can differ. It's always a good idea to research salaries in your specific geographical area and industry. 😊 Hope this helps!


In [31]:
# content = "What subjects should I study to be a data engineer?"

response = prompt_engineering(message_text)
print(response)

Hi there! 👋 Great to hear that you're interested in becoming a data engineer! This field is exciting and full of opportunities. 🚀 

Here are the key subjects you should focus on to become a competent data engineer:

1. **Computer Science Fundamentals**: Understanding algorithms, data structures, and computer networks is a must. 🖥️

2. **Programming**: Proficiency in languages like Python, Java, Scala, or R is crucial. Python is especially popular for data work. 🐍

3. **Databases**: Learn about both SQL (Structured Query Language) and NoSQL databases. Understand how to design, implement, and manage them. 🗄️

4. **Data Warehousing Solutions**: Familiarize yourself with data warehousing solutions like Amazon Redshift, Google BigQuery, or Snowflake. 🏬

5. **ETL (Extract, Transform, Load) Tools**: These tools are used to retrieve, clean, and load data. Examples include Informatica, Talend, and Microsoft SSIS. 🧹

6. **Big Data Technologies**: Hadoop, Spark, and Flink are key big data technol

In [34]:
# What is the difference between a data engineer and a data scientist?

response = prompt_engineering(message_text)
print(response)

Hello! 🤗 That's an excellent question! The difference between a data engineer and a data scientist can sometimes be a little blurry, but here's a basic breakdown:

1. Data Engineer 👷‍♀️: 
   Data engineers are the architects of data. They create and manage data infrastructure, which includes everything from databases to large-scale processing systems. They design, construct, install, test, and maintain highly scalable data management systems. They ensure that data is clean, available, and distributed efficiently for use in various applications. Their tools are often programming languages like Java, SQL, and Python, as well as systems like Hadoop and Spark.

2. Data Scientist 👩‍🔬: 
   Data scientists, on the other hand, are more like detectives. They use the infrastructure built by data engineers to analyze, interpret, and derive insights from data. They apply statistical analysis, predictive modeling, and machine learning techniques to business questions, and communicate their findings

# Tools prompt

In [ ]:
# there will be 4 tools
# Data Engineering tool, Machine Learning tool, Data Science tool, MLOps tool
# tools must have a dynamic prompt
# this means that the prompt for the four tools must be the same, but the keywords must be manipulated as variables

In [ ]:
# set a prompt template 

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    You are a tool focused on the {data_theme} theme. 
    Your role is to act as a friendly expert on the subject, 
    ready to assist and guide users with smiles and helpful insights. 
    Let’s make data fun and approachable!
    """
)
system = prompt_template.format(data_theme="MLOps")
system

In [45]:
# set a tool 

@tool
def chain_testing_prompt_tools(input_text) -> str:
    """
    Tool for the MLOps theme. 
    """
    prompt = ChatPromptTemplate.from_messages([
            ("system", system),
            ("user", "{input}")
        ])

    chain = prompt | model 

    return chain.invoke({"input": input_text})

In [46]:
response = chain_testing_prompt_tools.invoke("What is the salary of an MLOps Engineer?")
print(response.content)


The salary of an MLOps Engineer can vary greatly depending on factors such as location, level of experience, and the specific company. However, as a general range, in the United States, MLOps Engineers can expect to earn between $90,000 to $160,000 annually. In areas with a high cost of living or at particularly large tech companies, salaries can exceed this range. It's always best to research salaries in your specific area for the most accurate information.


In [ ]:
# MLOps 

"""
This specialized MLOps tool serves as your expert assistant 
for managing machine learning operations. 
It provides support by answering questions and guiding best practices for implementing 
and scaling ML models efficiently in production environments.
"""

In [ ]:
# DE

"""
Designed as an expert in data engineering, 
this tool offers guidance on efficient data management. 
It answers questions and assists in developing optimized data pipelines, 
facilitating effective data collection, integration, and processing
"""

In [ ]:
# DS

"""
Acting as a data science expert, this tool helps you navigate through data analysis, 
machine learning, and statistical modeling. 
It responds to inquiries, providing insights to transform raw data into 
strategic decisions through advanced analysis and visualization
"""

In [ ]:
# ME

"""
This tool specializes in machine learning engineering, 
supporting you from model design to evaluation. 
It answers questions about algorithm selection, hyperparameter tuning, 
and more, ensuring the development of efficient and effective ML solutions.
"""